In [13]:
import sys
import pandas as pd
import numpy as np
import re
from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import svm

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier



from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import GaussianNB

import nltk
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import GridSearchCV



In [25]:
t = pd.read_csv("./reports/data.csv")
t = t.dropna()
__type = "geometry"#sys.argv[1]
t[__type] =  t["labels"].str.contains(__type)
t[__type] = pd.to_numeric(t[__type], downcast="signed")

In [26]:
l =t['ps']= t['problem-statement'].tolist()
l2 = t['os']= t['output-spec'].tolist()
res1 = []
res2 = []
regex = re.compile("^a-zA-Z0-9\ '" )
def clean_s(input_str):
    return regex.sub(' ', input_str)

for i in range(len(l)):
    try:
        res1.append(clean_s(l[i]))
    except:
        res1.append("")
for i in range(len(l2)):
    try:
        res2.append(clean_s(l2[i]))
    except:
        res2.append("")

In [27]:

output = t[[__type,'ps','os' ]]
output = output.dropna()

output[__type] = output[__type].astype(int)

a = output.loc[output[__type] == 1.0]
b = output.loc[output[__type] == 0.0]
b = b.sample(n=len(a), random_state = 42)
output = a.append(b)


In [28]:
from __future__ import print_function
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import LSTM
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import hashing_trick

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.optimizers import Adam

max_features = 20000
maxlen = 400
batch_size = 32
embedding_dims = 500
filters = 250
kernel_size = 3
hidden_dims = 2500
epochs = 2


temp1 =  [hashing_trick(a, max_features) for a in output['ps'].tolist()]
temp2 =  [hashing_trick(a, max_features) for a in output['os'].tolist()]
pad1= sequence.pad_sequences(temp1);
pad2= sequence.pad_sequences(temp2);
temp = np.concatenate((sequence.pad_sequences(temp1), sequence.pad_sequences(temp2)), axis = 1) #np.concatenate((a, b.T), axis=1)
x_train, x_test, y_train, y_test = train_test_split(temp,output[__type], test_size=0.2,random_state=42)



In [29]:
# Convolution
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 70

model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Dropout(0.5))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))
y_pred = model.predict(x_test)
y_pred = [round(value) for value in y_pred]
print(classification_report(y_test, y_pred))

Train...
Train on 307 samples, validate on 77 samples
Epoch 1/10
307/307 [==============================] - 5s 15ms/step - loss: 0.6896 - acc: 0.5440 - val_loss: 0.6975 - val_acc: 0.4286
Epoch 2/10
307/307 [==============================] - 3s 11ms/step - loss: 0.6729 - acc: 0.6059 - val_loss: 0.6818 - val_acc: 0.6364
Epoch 3/10
307/307 [==============================] - 3s 11ms/step - loss: 0.6429 - acc: 0.6808 - val_loss: 0.6285 - val_acc: 0.6883
Epoch 4/10
307/307 [==============================] - 3s 11ms/step - loss: 0.5703 - acc: 0.7492 - val_loss: 0.5620 - val_acc: 0.6883
Epoch 5/10
307/307 [==============================] - 3s 11ms/step - loss: 0.4695 - acc: 0.8078 - val_loss: 0.5008 - val_acc: 0.7662
Epoch 6/10
307/307 [==============================] - 3s 11ms/step - loss: 0.3538 - acc: 0.8697 - val_loss: 0.4843 - val_acc: 0.7403
Epoch 7/10
307/307 [==============================] - 3s 11ms/step - loss: 0.2472 - acc: 0.9121 - val_loss: 0.5017 - val_acc: 0.7662
Epoch 8/10
307/